## ONNX?

In [5]:
import torch

# Load the entire model
model = torch.jit.load('/workspace/NeuralCellAutomataExp/conv_algorithm_scripted10k.pt')
model.eval()  # Set to evaluation mode


RecursiveScriptModule(
  original_name=ConvAlgorithm
  (conv_sequence): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Conv2d)
    (1): RecursiveScriptModule(original_name=ReLU)
    (2): RecursiveScriptModule(original_name=Conv2d)
  )
)

In [12]:
device='cuda'
dummy_input = get_init_grid(env_args).unsqueeze(0).to(device)
print(model(dummy_input).shape)

torch.Size([1, 16, 64, 64])


In [14]:
import torch.onnx

# Define the path to save the ONNX model
onnx_model_path = "/workspace/NeuralCellAutomataExp/model_10k.onnx"

# Export the model
torch.onnx.export(
    model,                      # The model to export
    dummy_input,                # An example input tensor
    onnx_model_path,            # Where to save the ONNX model
    export_params=True,         # Store the trained parameter weights inside the model file
    opset_version=12,           # ONNX version to export the model to (adjust as needed)
    do_constant_folding=True,   # Whether to execute constant folding for optimization
    input_names=['input'],      # The model's input names
    output_names=['output'],    # The model's output names
    dynamic_axes={
        'input': {0: 'batch_size'},    # Variable batch size
        'output': {0: 'batch_size'}
    }
)


## Testing equivalence

In [ ]:
with torch.no_grad():
    pytorch_output = model(dummy_input)

In [ ]:
import onnxruntime as ort
import numpy as np

ort_session = ort.InferenceSession("model.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(dummy_input)}
ort_outs = ort_session.run(None, ort_inputs)

onnx_output = ort_outs[0]
